# Horse or Human? In-graph training loop

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt

### Prepare the dataset

In [5]:
splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'], data_dir='./data')

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [6]:
BATCH_SIZE = 32
IMAGE_SIZE = 224

## Pre-process an image

In [7]:
@tf.function
def map_fn(img, label):
    image_height = 224
    image_width = 224
    img = tf.image.resize(img, (image_height, image_width))
    img /= 256
    return img, label

In [8]:
test_image, test_label = list(train_examples)[0]
test_result = map_fn(test_image, test_label)

print(test_result[0].shape)
print(test_result[1].shape)

del test_image, test_label, test_result

(224, 224, 3)
()


## Apply pre-processing to the datasets

In [9]:
def prepare_dataset(train_examples, validation_examples, test_examples, num_examples, map_fn, batch_size):
    train_ds = train_examples.map(map_fn).shuffle(num_examples).batch(batch_size)
    valid_ds = validation_examples.map(map_fn).batch(batch_size)
    test_ds = test_examples.map(map_fn).batch(batch_size)
    return train_ds, valid_ds, test_ds

In [10]:
train_ds, valid_ds, test_ds = prepare_dataset(train_examples, validation_examples, test_examples, num_examples, map_fn, BATCH_SIZE)

In [ ]:
test_train_ds = list(train_ds)
print(len(test_train_ds))
print(test_train_ds[0][0].shape)

del test_train_ds

### Define the model

In [ ]:
MODULE_HANDLE = 'data/resnet_50_feature_vector'
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()

## Define optimizer

In [ ]:
def set_adam_optimizer():
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    return optimizer

In [ ]:
test_optimizer = set_adam_optimizer()

print(type(test_optimizer))

del test_optimizer

## Define the loss function

In [ ]:
def set_sparse_cat_crossentropy_loss():
    train_loss = tf.keras.losses.SparseCategoricalCrossentropy()
    val_loss = tf.keras.losses.SparseCategoricalCrossentropy()
    return train_loss, val_loss

In [ ]:
test_train_loss, test_val_loss = set_sparse_cat_crossentropy_loss()

print(type(test_train_loss))
print(type(test_val_loss))

del test_train_loss, test_val_loss

## Define the acccuracy function

In [ ]:
def set_sparse_cat_crossentropy_accuracy():
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    return train_accuracy, val_accuracy

In [ ]:
test_train_accuracy, test_val_accuracy = set_sparse_cat_crossentropy_accuracy()

print(type(test_train_accuracy))
print(type(test_val_accuracy))

del test_train_accuracy, test_val_accuracy

In [ ]:
optimizer = set_adam_optimizer()
train_loss, val_loss = set_sparse_cat_crossentropy_loss()
train_accuracy, val_accuracy = set_sparse_cat_crossentropy_accuracy()

### Define the training loop

In [ ]:
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'
EPOCHS = 2

def train_one_step(model, optimizer, x, y, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = train_loss(y, predictions)
    grads = tape.gradient(loss, model.weights)
    optimizer.apply_gradients(zip(grads, model.weights))
    train_accuracy(y, predictions)
    return loss

In [ ]:
def base_model():
    inputs = tf.keras.layers.Input(shape=(2))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

test_model = base_model()

test_optimizer = set_adam_optimizer()
test_image = tf.ones((2,2))
test_label = tf.ones((1,))
test_train_loss, _ = set_sparse_cat_crossentropy_loss()
test_train_accuracy, _ = set_sparse_cat_crossentropy_accuracy()

test_result = train_one_step(test_model, test_optimizer, test_image, test_label, test_train_loss, test_train_accuracy)
print(test_result)

del test_result, test_model, test_optimizer, test_image, test_label, test_train_loss, test_train_accuracy

## Define the 'train' function

In [ ]:
@tf.function
def train(model, optimizer, epochs, device, train_ds, train_loss, train_accuracy, valid_ds, val_loss, val_accuracy):
    step = 0
    loss = 0.0
    accuracy = 0.0
    for epoch in range(epochs):
        for x, y in train_ds:
            step += 1
            with tf.device(device_name=device):
                loss = train_one_step(model, optimizer, x, y, train_loss, train_accuracy)
            tf.print('Step', step, 
                   ': train loss', loss, 
                   '; train accuracy', train_accuracy.result())
        with tf.device(device_name=device):
            for x, y in valid_ds:
                y_pred = model(x)
                loss = val_loss(y, y_pred)
                accuracy = val_accuracy(y, y_pred)
        tf.print('val loss', loss, '; val accuracy', accuracy)

In [ ]:
train(model, optimizer, EPOCHS, device, train_ds, train_loss, train_accuracy, valid_ds, val_loss, val_accuracy)

# Evaluation

In [ ]:
test_imgs = []
test_labels = []

predictions = []
with tf.device(device_name=device):
    for images, labels in test_ds:
        preds = model(images)
        preds = preds.numpy()
        predictions.extend(preds)

        test_imgs.extend(images.numpy())
        test_labels.extend(labels.numpy())

In [ ]:
class_names = ['horse', 'human']

def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    img = np.squeeze(img)
    plt.imshow(img, cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions_array)

    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    
    print(true_label)
  
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

### Plot the result of a single image

In [ ]:
index = 8 
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(index, predictions, test_labels, test_imgs)
plt.show()

## References
#### Coursera: Custom and Distributed Training with TensorFlow [course](https://www.coursera.org/learn/custom-distributed-training-with-tensorflow).